## Add project root

In [ ]:
import sys
from pathlib import Path
import importlib

# Add the parent of the *outer* DLC-Jupyter-Notebooks folder to sys.path
project_root = Path().resolve().parents[0]
print(project_root)
sys.path.append(str(project_root))

## Connect to database

In [ ]:
from Python_scripts import config
import pandas as pd
import platform
import os

try:
    conn = config.get_conn()
    cursor = conn.cursor()
    use_csv_fallback = False
    print("[INFO] Connected to Postgres")
except Exception as exc:
    print(f"[WARN] DB connection failed, falling back to CSVs: {exc}")
    conn = None
    cursor = None
    use_csv_fallback = True
    data_dir = config.get_data_dir()
    print(f"[INFO] Using DATA_DIR = {data_dir}")
    
    dlc_tables = {}
    if data_dir.exists():
        for p in data_dir.glob('dlc_table_*.csv'):
            try:
                dlc_tables[p.name] = pd.read_csv(p)
                print(f"[INFO] Loaded {p.name}")
            except Exception as e:
                print(f"[WARN] Couldn't load {p}: {e}")
    else:
        print(f"[WARN] DATA_DIR does not exist: {data_dir}")

## Import modules

In [ ]:
import importlib
import Python_scripts.Data_analysis.fetch_id_list as fetch_mod
importlib.reload(fetch_mod)
from Python_scripts.Data_analysis.fetch_id_list import fetch_id_list

In [ ]:
import importlib
import Python_scripts.Feature_functions.motion_features
import Python_scripts.Feature_functions.trajectory_curvature
import Python_scripts.Feature_functions.angle_features
import Python_scripts.Data_analysis.plot_groupwise_bar

importlib.reload(Python_scripts.Feature_functions.motion_features)
importlib.reload(Python_scripts.Feature_functions.trajectory_curvature)
importlib.reload(Python_scripts.Feature_functions.angle_features)
importlib.reload(Python_scripts.Data_analysis.plot_groupwise_bar)

from Python_scripts.Feature_functions.motion_features import (
    compute_motion_features_per_minute, batch_compute_motion_features_per_minute
)
from Python_scripts.Feature_functions.trajectory_curvature import (
    compute_trajectory_curvature, batch_trajectory_curvature
)
from Python_scripts.Feature_functions.angle_features import (
    angle_features_for_trial, batch_angle_features
)
from Python_scripts.Data_analysis.plot_groupwise_bar import plot_groupwise_bar

## Fetch trial IDs (2X dose)

In [ ]:
task_name = "AllTask"

saline_id, ghrelin_id, exc_id, inh_id = fetch_id_list(
    conn,
    task_name=None,  # None = AllTask, or specify ['FoodLight', 'ToyOnly', 'ToyLight', 'LightOnly']
    dose_mult=2,
    genotype="white",
    csv_prefix="dlc_table",
    min_trial_length=None
)

print(f"saline_id: {len(saline_id)} trials")
print(f"ghrelin_id: {len(ghrelin_id)} trials")
print(f"exc_id: {len(exc_id)} trials")
print(f"inh_id: {len(inh_id)} trials")

---
# VELOCITY ANALYSIS

## Velocity: Parameter sweep

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

bodypart = 'Head'
time_limit = None
smooth = True
window_range = range(5, 26, 5)

group_specs = {
    "Saline": saline_id,
    "Ghrelin": ghrelin_id,
    "Excitatory": exc_id,
    "Inhibitory": inh_id
}
group_specs = {label: ids for label, ids in group_specs.items() if ids not in (None, [], ())}

outfile = f"White_2X_{task_name}_{bodypart}_velocity_window_sweep.pdf"
with PdfPages(outfile) as pdf:
    for window_size in window_range:
        print(f"[INFO] Analyzing window_size = {window_size}")
        
        frames = []
        for label, ids in group_specs.items():
            df = batch_compute_motion_features_per_minute(
                conn, ids,
                bodypart=bodypart,
                time_limit=time_limit,
                smooth=smooth,
                window=window_size,
            )
            df = df[['trial_id', 'velocity_per_min']].copy().dropna()
            df['group'] = label
            frames.append(df)
        
        if not frames:
            print("[WARN] No groups provided—skipping this window.")
            continue
        
        df_vel = pd.concat(frames, ignore_index=True)
        
        order = list(group_specs.keys())
        fig, ax, stats_df = plot_groupwise_bar(
            df_vel,
            y='velocity_per_min',
            ylabel='Average speed (units/min)',
            plot_type='bar',
            show_points=False,
            order=order,
            show_stats=True,
            tests_to_show=("ttest",)
        )
        ax.set_title(f"{task_name} | {bodypart} | window={window_size}", pad=20)
        
        pdf.savefig(fig, bbox_inches='tight')
        plt.close(fig)

print(f"[✓] Saved {outfile}")

## Velocity: Single analysis with fixed parameters

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

bodypart = 'Head'
time_limit = None
smooth = False
window = 5

group_specs = {
    "Saline": saline_id,
    "Ghrelin": ghrelin_id,
    "Excitatory": exc_id,
    "Inhibitory": inh_id
}
group_specs = {label: ids for label, ids in group_specs.items() if ids not in (None, [], ())}

frames = []
for label, ids in group_specs.items():
    df = batch_compute_motion_features_per_minute(
        conn, ids,
        bodypart=bodypart,
        time_limit=time_limit,
        smooth=smooth,
        window=window,
    )
    df = df[['trial_id', 'velocity_per_min']].copy()
    df['group'] = label
    frames.append(df)

if not frames:
    raise ValueError("[WARN] No groups provided—nothing to plot.")

df_vel = pd.concat(frames, ignore_index=True).dropna(subset=['velocity_per_min'])

order = list(group_specs.keys())
fig, ax, stats_df = plot_groupwise_bar(
    df_vel, y='velocity_per_min',
    ylabel='Average speed (units/min)',
    plot_type='bar', show_points=True,
    order=order, show_stats=True,
    tests_to_show=("ttest",)
)

In [ ]:
print(stats_df)

In [ ]:
ax.set_title(f"{task_name} | {bodypart} | window={window}", pad=20)
fig.savefig(f"White_2X_{task_name}_velocity.pdf", dpi=300, bbox_inches='tight')

## Velocity: Export to Excel

In [ ]:
trial_ids = df_vel['trial_id'].unique().tolist()

placeholders = ','.join(['%s'] * len(trial_ids))
query = f"""
SELECT id, task, modulation
FROM dlc_table
WHERE id IN ({placeholders});
"""
meta_df = pd.read_sql_query(query, conn, params=tuple(trial_ids))

df_out = df_vel.merge(meta_df, left_on='trial_id', right_on='id', how='left')
df_out = df_out.drop(columns=['id'])

df_out.to_excel("2X_White_Velocity.xlsx", index=False)
print("[✓] Saved 2X_White_Velocity.xlsx")

---
# CURVATURE ANALYSIS

## Curvature: Parameter sweep

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

bodypart = 'Midback'
window_range = range(11, 28, 4)

group_specs = {
    "Saline": saline_id,
    "Ghrelin": ghrelin_id,
    "Excitatory": exc_id,
    "Inhibitory": inh_id
}
group_specs = {label: ids for label, ids in group_specs.items() if ids not in (None, [], ())}

outfile = f"White_2X_{task_name}_curvature_window_sweep.pdf"
with PdfPages(outfile) as pdf:
    for window_size in window_range:
        print(f"[INFO] Analyzing window_size = {window_size}")
        
        frames = []
        for label, ids in group_specs.items():
            df = batch_trajectory_curvature(
                conn, ids, bodypart=bodypart, window=window_size
            )
            df["group"] = label
            frames.append(df)
        
        if not frames:
            print("[WARN] No groups provided—skipping this window.")
            continue
        
        df_curv = pd.concat(frames, ignore_index=True)
        
        order = list(group_specs.keys())
        fig, ax, stats_df = plot_groupwise_bar(
            df_curv, y='mean_curvature',
            ylabel='Mean curvature',
            plot_type='bar', show_points=False,
            order=order, show_stats=True,
            tests_to_show=("ttest",)
        )
        
        ax.set_title(f"{task_name} | window size = {window_size:d}", pad=20)
        
        pdf.savefig(fig, bbox_inches='tight')
        plt.close(fig)

print(f"[✓] Saved {outfile}")

## Curvature: Single analysis with fixed parameters

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

bodypart = 'Midback'
window_size = 23

group_specs = {
    "Saline": saline_id,
    "Ghrelin": ghrelin_id,
    "Excitatory": exc_id,
    "Inhibitory": inh_id
}
group_specs = {label: ids for label, ids in group_specs.items() if ids not in (None, [], ())}

frames = []
for label, ids in group_specs.items():
    df = batch_trajectory_curvature(
        conn, ids, bodypart=bodypart, window=window_size
    )
    df["group"] = label
    frames.append(df)

if not frames:
    print("[WARN] No groups provided—skipping this window.")
    raise ValueError("No data")

df_curv = pd.concat(frames, ignore_index=True)

order = list(group_specs.keys())
fig, ax, stats_df = plot_groupwise_bar(
    df_curv, y='mean_curvature',
    ylabel='Mean curvature',
    plot_type='bar', show_points=True,
    order=order, show_stats=True,
    tests_to_show=("ttest",)
)

In [ ]:
print(stats_df)

In [ ]:
ax.set_title(f"{task_name} | window size = {window_size:d}", pad=20)
fig.savefig(f"White_2X_{task_name}_curvature.pdf", dpi=300, bbox_inches='tight')

## Curvature: Export to Excel

In [ ]:
trial_ids = df_curv['id'].unique().tolist()

query = """
SELECT id, task, modulation
FROM dlc_table
WHERE id = ANY(%s);
"""
meta_df = pd.read_sql_query(query, conn, params=(trial_ids,))

df_out = df_curv.merge(meta_df, left_on='id', right_on='id', how='left')

df_out.to_excel("2X_White_Curvature.xlsx", index=False)
print("[✓] Saved 2X_White_Curvature.xlsx")

---
# ANGLE FEATURES ANALYSIS

## Angle: Parameter sweep

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

likelihood_thr = np.arange(0.65, 0.68, 0.025)
window_size = None

group_specs = {
    "Saline": saline_id,
    "Ghrelin": ghrelin_id,
    "Excitatory": exc_id,
    "Inhibitory": inh_id
}
group_specs = {label: ids for label, ids in group_specs.items() if ids not in (None, [], ())}

outfile = f"White_2X_{task_name}_ang_likelihood_sweep.pdf"
with PdfPages(outfile) as pdf:
    for likelihood in likelihood_thr:
        print(f"[INFO] Analyzing likelihood = {likelihood}")
        
        frames = []
        for label, ids in group_specs.items():
            df = batch_angle_features(
                conn, ids, likelihood_threshold=likelihood, smooth_window=window_size,
            )
            df = df[['trial_id', 'head_body_misalignment_p95']].copy().dropna()
            df['group'] = label
            frames.append(df)
        
        if not frames:
            print("[WARN] No groups provided—skipping this window.")
            continue
        
        df = pd.concat(frames, ignore_index=True)
        
        order = list(group_specs.keys())
        fig, ax, stats_df = plot_groupwise_bar(
            df,
            y='head_body_misalignment_p95',
            ylabel='head_body_misalignment_p95',
            plot_type='bar',
            show_points=False,
            order=order,
            show_stats=True,
            tests_to_show=("ttest",)
        )
        ax.set_title(f"{task_name} | likelihood={likelihood}", pad=20)
        
        pdf.savefig(fig, bbox_inches='tight')
        plt.close(fig)

print(f"[✓] Saved {outfile}")

## Angle: Single analysis with fixed parameters

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

smooth_window = None
likelihood_threshold = 0.65

group_specs = {
    "Saline": saline_id,
    "Ghrelin": ghrelin_id,
    "Excitatory": exc_id,
    "Inhibitory": inh_id
}
group_specs = {label: ids for label, ids in group_specs.items() if ids not in (None, [], ())}

frames = []
for label, ids in group_specs.items():
    df = batch_angle_features(
        conn, ids, likelihood_threshold=likelihood_threshold, smooth_window=smooth_window
    )
    df = df[['trial_id', 'head_body_misalignment_p95']].copy().dropna()
    df['group'] = label
    frames.append(df)

if not frames:
    print("[WARN] No groups provided—skipping this window.")
    raise ValueError("No data")

df = pd.concat(frames, ignore_index=True)

order = list(group_specs.keys())

fig, ax, stats_df = plot_groupwise_bar(
    df, y='head_body_misalignment_p95',
    ylabel='Mean head_body_misalignment_p95',
    plot_type='bar', show_points=True,
    order=order, show_stats=True,
    tests_to_show=("ttest",)
)

In [ ]:
print(stats_df)

In [ ]:
ax.set_title(f"{task_name} | likelihood_threshold={likelihood_threshold}", pad=20)
fig.savefig(f"White_2X_{task_name}_ang.pdf", dpi=300, bbox_inches='tight')

## Angle: Export to Excel

In [ ]:
trial_ids = df['trial_id'].unique().tolist()

if conn is not None:
    try:
        conn.rollback()
    except:
        pass
    
    placeholders = ','.join(['%s'] * len(trial_ids))
    query = f"""
    SELECT id, task, modulation
    FROM dlc_table
    WHERE id IN ({placeholders});
    """
    meta_df = pd.read_sql(query, conn, params=tuple(trial_ids))
else:
    csv_frames = []
    for name, table_df in (dlc_tables.items() if 'dlc_tables' in globals() else []):
        if isinstance(table_df, pd.DataFrame):
            csv_frames.append(table_df)
    if csv_frames:
        meta_all = pd.concat(csv_frames, ignore_index=True)
        meta_df = meta_all[meta_all['id'].isin(trial_ids)][['id', 'task', 'modulation']].drop_duplicates()
    else:
        meta_df = pd.DataFrame(columns=['id', 'task', 'modulation'])

if 'id' in meta_df.columns:
    df_out = df.merge(meta_df, left_on='trial_id', right_on='id', how='left')
    df_out = df_out.drop(columns=['id']) if 'id' in df_out.columns else df_out
else:
    df_out = df.copy()

df_out.to_excel("2X_White_Angle.xlsx", index=False)
print("[✓] Saved 2X_White_Angle.xlsx")